<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c8bcd13cf23afdf93eb03c1771d89f728f501a80fa4eb70779cc5d3cd3779019
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-29 14:22:21
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -94.00 K (-0.67%)
Current PnL: -26.81 L (-17.32%)
CY Booked + Current PnL: -12.52 L (-8.09%)
-------------------
Total profit:  1.46 L
Total loss:  -28.27 L
-------------------
Total Booked + Current PnL: 14.46 L (11.31%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.19%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.44 L (65.2%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.06%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.94,-19.52,24.37,0.09,19765.0,-19675.0,81102.0,88.26,38.0,M-SC,2.93,253.0,-1.00,0.58,6.55,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.68,8.78,10.03,19.69,20920.0,16833.0,208575.0,-1.77,55.0,X-MC,2.40,78.0,0.80,1.49,16.17,XY25,NTT,AC
52,NESTLEIND,1377.00,0.20,12.41,7.99,21.40,23840.0,32947.0,298373.0,4.29,63.0,X-LC,5.52,12.0,1.38,2.14,19.53,XY25,NTT,FMCG
50,MASFIN,398.61,-1.12,-4.38,27.64,22.05,25896.0,-4290.0,93690.0,-17.34,47.0,H-SC,6.52,164.0,-0.17,0.67,36.19,XR,ATH,FINANCE
43,ITC,452.00,-0.38,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,48.0,X-LC,2.05,5.0,-0.11,1.70,3.84,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.0,3.49,-38.23,119.00,35.27,83464.0,-43411.0,70138.0,-51.71,54.0,L-SC,8.78,270.0,-0.52,0.50,71.66,XR,NTT,HOTELS
57,RAJOOENG,143.1,3.09,-37.86,124.68,39.61,79409.0,-38810.0,63690.0,-57.95,36.0,H-SC,31.75,135.0,-0.49,0.46,3.09,AR,ATH,MISC
44,JCHAC,2282.0,1.47,-34.82,53.40,-0.01,44485.0,-44499.0,83306.0,16943.48,42.0,M-SC,12.53,233.0,-1.00,0.60,6.62,OX40N,NTT,AC
63,SFL,1287.0,1.45,-42.17,118.95,26.63,180390.0,-110566.0,151652.0,14.87,44.0,M-SC,12.41,239.0,-0.61,1.09,3.82,XY24,NTT,MISC
74,TITAGARH,1548.0,1.23,-14.44,70.33,45.74,145728.0,-34971.0,207206.0,13.90,71.0,H-SC,5.81,174.0,-0.24,1.48,34.77,XY24,NTT,ENGINEERING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
25,FINCABLES,1641.55,-3.38,-6.11,121.44,107.91,168344.0,-9021.0,138623.0,-23.79,41.0,M-SC,11.70,216.0,-0.05,0.99,3.36,OX40N,ATH,CABLES
22,DIXON,18931.72,-3.27,-20.59,58.76,26.07,91093.0,-40196.0,155025.0,-61.38,25.0,X-MC,14.01,56.0,-0.44,1.11,0.00,X40N,ATH,IT
13,BSOFT,831.70,-2.80,-20.92,90.67,50.79,100853.0,-29422.0,111231.0,1.28,58.0,H-SC,9.96,151.0,-0.29,0.80,29.51,XR,ATH,IT
69,SURYODAY,216.00,-2.65,-20.43,54.62,23.03,77830.0,-36577.0,142494.0,55.70,51.0,H-SC,6.86,167.0,-0.47,1.02,41.38,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.08,-0.9,83.33,81.67,77467.0,-846.0,92964.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.03,XR,NTT,CERAMICS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-3.38,-6.11,121.44,107.91,168344.0,-9021.0,138623.0,-23.79,41.0,M-SC,11.70,216.0,-0.05,0.99,3.36,OX40N,ATH,CABLES
77,TTKPRESTIG,770.00,-0.94,-19.52,24.37,0.09,19765.0,-19675.0,81102.0,88.26,38.0,M-SC,2.93,253.0,-1.00,0.58,6.55,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.04,-19.88,24.83,0.02,34727.0,-34699.0,139860.0,113.17,33.0,M-SC,9.82,234.0,-1.00,1.00,21.65,OX40N,NTT,PAINTS
47,KANSAINER,340.00,-1.33,-21.82,45.14,13.47,95170.0,-58833.0,210834.0,-67.92,54.0,H-SC,6.83,158.0,-0.62,1.51,7.20,XY24,NTT,PAINTS
66,SIS,528.00,-1.27,-23.04,57.56,21.26,49185.0,-25582.0,85450.0,2017.62,55.0,H-SC,4.77,166.0,-0.52,0.61,15.45,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.6,8.57,58.67,72.27,107933.0,14518.0,183966.0,-6.99,60.0,M-LC,4.44,99.0,0.13,1.32,15.27,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.60,8.57,58.67,72.27,107933.0,14518.0,183966.0,-6.99,60.0,M-LC,4.44,99.0,0.13,1.32,15.27,XR,NTT,IT
6,ASIANTILES,137.00,-2.08,-0.90,83.33,81.67,77467.0,-846.0,92964.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.03,XR,NTT,CERAMICS
38,INDIAMART,4810.62,0.13,-4.64,116.79,106.72,137353.0,-5729.0,117607.0,-52.73,39.0,H-SC,5.05,139.0,-0.04,0.84,17.65,AR,ATH,MISC
25,FINCABLES,1641.55,-3.38,-6.11,121.44,107.91,168344.0,-9021.0,138623.0,-23.79,41.0,M-SC,11.70,216.0,-0.05,0.99,3.36,OX40N,ATH,CABLES
1,ABB,7934.00,-0.28,-5.17,53.51,45.57,132752.0,-13531.0,248088.0,-39.05,50.0,H-MC,4.53,120.0,-0.10,1.78,5.56,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-3.27,-20.59,58.76,26.07,91093.0,-40196.0,155025.0,-61.38,25.0,X-MC,14.01,56.0,-0.44,1.11,0.00,X40N,ATH,IT
9,BAJAJHFL,181.50,-1.14,-19.44,92.98,55.47,182241.0,-47286.0,196000.0,-28.68,26.0,X-MC,16.55,64.0,-0.26,1.40,0.43,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.18,-19.09,77.14,43.32,101655.0,-31092.0,131780.0,-30.52,28.0,X-SC,6.60,90.0,-0.31,0.94,1.18,X40N,ATH,ELECTRICAL
8,AWL,485.00,-0.32,-25.25,105.33,53.50,237594.0,-76180.0,225571.0,-63.76,30.0,X-MC,6.83,58.0,-0.32,1.62,0.19,XY24,NTT,FMCG
3,ACC,3906.00,-0.65,-27.55,126.57,64.15,218207.0,-65551.0,172400.0,-57.62,30.0,X-SC,4.81,82.0,-0.30,1.24,0.00,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.36,-3.09,25.00,21.14,55080.0,-7019.0,220320.0,-9.95,47.0,X-MC,1.22,74.0,-0.13,1.58,22.35,XY24,NTT,PAINTS
78,UNITDSPR,1644.0,-0.45,4.46,15.61,20.77,38180.0,10440.0,244584.0,-4.59,49.0,X-MC,1.70,62.0,0.27,1.75,11.69,X40N,NTT,BREWERIES
15,CAMS,950.0,-1.04,-2.91,28.21,24.48,68982.0,-7326.0,244530.0,-81.54,41.0,X-SC,1.89,86.0,-0.11,1.75,19.56,X40N,NTT,MISC
65,SIEMENS,4671.5,-1.07,-17.58,52.28,25.51,80190.0,-32710.0,153385.0,-5.83,39.0,H-LC,1.96,49.0,-0.41,1.10,13.03,AR,ATH,ELECTRICAL
43,ITC,452.0,-0.38,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,48.0,X-LC,2.05,5.0,-0.11,1.70,3.84,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.65,-27.55,126.57,64.15,218207.0,-65551.0,172400.0,-57.62,30.0,X-SC,4.81,82.0,-0.30,1.24,0.00,XY24,BTT,CEMENT
19,COLPAL,3726.84,-1.15,-21.65,80.62,41.51,166351.0,-57025.0,206340.0,-6.75,32.0,X-MC,8.18,61.0,-0.34,1.48,0.00,XY25,ATH,FMCG
22,DIXON,18931.72,-3.27,-20.59,58.76,26.07,91093.0,-40196.0,155025.0,-61.38,25.0,X-MC,14.01,56.0,-0.44,1.11,0.00,X40N,ATH,IT
8,AWL,485.00,-0.32,-25.25,105.33,53.50,237594.0,-76180.0,225571.0,-63.76,30.0,X-MC,6.83,58.0,-0.32,1.62,0.19,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.91,-39.24,122.51,35.20,95786.0,-50484.0,78186.0,2.53,33.0,X-SC,17.25,92.0,-0.53,0.56,0.29,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.97,-29.16,101.65,42.84,46809.0,-18957.0,46049.0,-53.72,46.0,X-SC,33.01,83.0,-0.40,0.33,4.18,XY24,NTT,MISC
59,RELAXO,1176.00,-0.79,-48.20,194.00,52.31,145888.0,-69960.0,75200.0,-44.83,40.0,X-SC,7.32,91.0,-0.48,0.54,0.66,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.91,-39.24,122.51,35.20,95786.0,-50484.0,78186.0,2.53,33.0,X-SC,17.25,92.0,-0.53,0.56,0.29,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.28,-3.14,28.10,24.08,34552.0,-3990.0,122960.0,-10.94,41.0,X-SC,6.96,89.0,-0.12,0.88,15.86,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.18,-19.09,77.14,43.32,101655.0,-31092.0,131780.0,-30.52,28.0,X-SC,6.60,90.0,-0.31,0.94,1.18,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-0.92,-10.65,35.09,20.71,118593.0,-40268.0,337969.0,-22.98,56.0,X-LC,3.00,1.0,-0.34,2.42,12.93,X40,ATH,IT
41,INFY,1972.00,-0.67,11.98,19.88,34.24,69657.0,37497.0,350385.0,-12.78,61.0,X-LC,6.73,2.0,0.54,2.51,21.26,X40,NTT,IT
75,TMPV,600.00,-0.10,-26.53,67.39,22.98,111842.0,-59936.0,165962.0,-24.30,49.0,X-LC,6.13,3.0,-0.54,1.19,4.38,XY24,NTT,AUTO
81,VBL,671.64,0.03,-2.43,39.06,35.67,120365.0,-7688.0,308154.0,-14.05,57.0,X-LC,3.21,4.0,-0.06,2.21,11.08,X40N,ATH,FMCG
43,ITC,452.00,-0.38,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,48.0,X-LC,2.05,5.0,-0.11,1.70,3.84,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.0,-1.24,-38.03,114.45,32.89,54756.0,-29364.0,47843.0,-690.94,49.0,L-MC,6.76,259.0,-0.54,0.34,31.76,XR,NTT,BANKS
56,RAJESHEXPO,518.0,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.0,-2.08,-0.90,83.33,81.67,77467.0,-846.0,92964.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.03,XR,NTT,CERAMICS
13,BSOFT,831.7,-2.80,-20.92,90.67,50.79,100853.0,-29422.0,111231.0,1.28,58.0,H-SC,9.96,151.0,-0.29,0.80,29.51,XR,ATH,IT
69,SURYODAY,216.0,-2.65,-20.43,54.62,23.03,77830.0,-36577.0,142494.0,55.70,51.0,H-SC,6.86,167.0,-0.47,1.02,41.38,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.08,-0.90,83.33,81.67,77467.0,-846.0,92964.0,8372.22,67.0,L-SC,14.98,271.0,-0.01,0.67,80.03,XR,NTT,CERAMICS
13,BSOFT,831.7,-2.80,-20.92,90.67,50.79,100853.0,-29422.0,111231.0,1.28,58.0,H-SC,9.96,151.0,-0.29,0.80,29.51,XR,ATH,IT
74,TITAGARH,1548.0,1.23,-14.44,70.33,45.74,145728.0,-34971.0,207206.0,13.90,71.0,H-SC,5.81,174.0,-0.24,1.48,34.77,XY24,NTT,ENGINEERING
56,RAJESHEXPO,518.0,-5.00,-60.95,156.25,0.07,83703.0,-83607.0,53570.0,1863.38,52.0,L-SC,4.58,268.0,-1.00,0.38,33.23,OX40N,NTT,JEWELLERY
84,WIPRO,420.0,-0.60,8.57,58.67,72.27,107933.0,14518.0,183966.0,-6.99,60.0,M-LC,4.44,99.0,0.13,1.32,15.27,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.40
1,25,44.96
2,50,76.65


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.90
MC    30.78
LC    24.31
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.90
X40      23.11
X40N     12.85
XR        9.82
XY25      9.21
AR        8.85
OX40N     7.53
SR        0.99
MH        0.73
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.20
H-SC    23.84
X-LC    20.83
M-SC    11.48
X-SC     8.03
H-MC     4.84
L-SC     1.55
M-MC     1.40
M-LC     1.32
H-LC     1.10
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.41,-7.46,42.01
IT,12.92,-16.54,75.13
FINANCE,9.62,-18.59,70.16
MISC,7.18,-29.59,82.60
ELECTRICAL,5.95,-13.04,54.45
PAINTS,5.84,-13.24,30.06
INSURANCE,4.69,-3.36,38.74
PHARMA,4.07,-3.47,36.04
AUTO,2.84,-32.27,78.13


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3165923.0,21
AR,1337846.0,10
XR,1293196.0,13
X40,995721.0,14
X40N,891177.0,9
OX40N,740335.0,10
XY25,356019.0,6
SR,282421.0,2
MH,81840.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3651085.0,25
M-SC,1455719.0,15
X-MC,1448009.0,16
X-LC,819474.0,11
X-SC,798021.0,8
H-MC,403115.0,3
L-SC,244634.0,3
M-LC,107933.0,1
H-LC,80190.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1252591.0      6
           AR         912690.0      5
M-SC       XY24       823097.0      6
H-SC       XR         791629.0      7
X-MC       X40        490628.0      7
           XY24       412400.0      3
           X40N       357710.0      4
H-SC       OX40N      329914.0      4
M-SC       OX40N      326718.0      5
X-LC       X40        323165.0      5
X-SC       X40N       316525.0      3
           XY24       299568.0      3
H-SC       SR         282421.0      2
H-MC       AR         217009.0      2
X-LC       X40N       216942.0      2
           XY24       192161.0      2
X-MC       XY25       187271.0      2
H-MC       XY24       186106.0      1
X-SC       X40        181928.0      2
M-SC       XR         177947.0      2
L-SC       XR         160931.0      2
M-SC       AR         127957.0      2
M-LC       XR         107933.0      1
X-LC       XY25        87206.0      2
L-SC       OX40N       83703.0      1
H-SC       MH          81840.0      1
H-LC       AR          80190.0      1
L-MC       XR          54756.0      1
M-MC       XY25        50151.0      1
L-LC       XY25        31391.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
